In [104]:
import fastf1
import fastf1.core
import fastf1.events
import plotly.express as px
from fastf1.plotting import get_compound_color, get_driver_color, list_compounds
import numpy as np
import pandas as pd

telemetry_metrics = ["Speed", "RPM", "Throttle", "Brake", "nGear"]
position_metrics = ["X", "Y", "Z", "Time"]

In [105]:
events = fastf1.get_event_schedule(year=2024)
round = events.get_event_by_round(1)
sessions = round.get_session('Race')

sessions.load()

sessions

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']


2024 Season Round 1: Bahrain Grand Prix - Race

In [106]:
def angle_between(v1, v2):
    dot_product = np.dot(v1, v2)
    mag1 = np.linalg.norm(v1)
    mag2 = np.linalg.norm(v2)
    cos_theta = dot_product / (mag1 * mag2)
    angle = np.arccos(cos_theta)
    angle = np.degrees(angle)

    return angle

def create_vector_sets(data: fastf1.core.Telemetry):
    data["dx"] = data["X"].diff()
    data["dy"] = data["Y"].diff()
    data["dirX"] = [None] + [
        data["X"].iloc[i] - data["X"].iloc[i - 1] for i in range(1, len(data["X"]))
    ]
    data["dirY"] = [None] + [
        data["Y"].iloc[i] - data["Y"].iloc[i - 1] for i in range(1, len(data["Y"]))
    ]
    data["BaseAngle"] = [None] + [angle_between([data["dirX"].iloc[1], data["dirY"].iloc[1]], [data["dirX"].iloc[i], data["dirY"].iloc[i]]) for i in range(1, len(data["dirX"]))]

    data["DeltaBaseAngle"] = data["BaseAngle"].diff()

    thetas = []
    for i in range(2, len(data["dirX"])):
        assert len(data["dirX"]) == len(data["dirY"])
        vec1 = np.array([data["dirX"].iloc[i], data["dirY"].iloc[i]])
        vec2 = np.array([data["dirX"].iloc[i - 1], data["dirY"].iloc[i - 1]])

        dot_product = np.dot(vec1, vec2)
        mag1 = np.linalg.norm(vec1)
        mag2 = np.linalg.norm(vec2)
        cos_theta = dot_product / (mag1 * mag2)
        angle = np.arccos(cos_theta)
        angle = np.degrees(angle)

        thetas.append(angle)

    data["angles"] = [None, None] + thetas

    print(data[["Time", "X", "Y", "dirX", "dirY", "angles", "BaseAngle"]])

    return data

In [107]:
car_data = sessions.laps.pick_drivers('VER').pick_wo_box()
final_frame = None
for lap_tup in car_data.iterlaps():
    lap = lap_tup[1]
    lap_num = lap["LapNumber"]
    print(f"Processing lap {lap_num}")
    telemetry = lap.get_telemetry()
    telemetry["LapNumber"] = lap_num
    final_frame = telemetry if final_frame is None else pd.concat([final_frame, telemetry])

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


Processing lap 1.0
Processing lap 2.0


core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


Processing lap 3.0
Processing lap 4.0


core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


...

Processing lap 51.0
Processing lap 52.0


core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


Processing lap 53.0
Processing lap 54.0


core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


Processing lap 55.0
Processing lap 56.0
Processing lap 57.0


In [108]:


final_frame
# car_data

speed_data = final_frame[["Speed", "RPM", "Throttle", "Brake", "nGear", "X", "Y", "Z", "Time", "LapNumber", "Source"]]
speed_data["Time"] = speed_data["Time"].apply(lambda x: x.total_seconds())
speed_data = create_vector_sets(speed_data[speed_data["Source"]== "car"])
speed_data_graph = speed_data[speed_data["LapNumber"] == 19]

speed_data_graph



/var/folders/7v/5sz92nkn6s13fvy6c8bz7_sm0000gn/T/ipykernel_26709/1897838449.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/7v/5sz92nkn6s13fvy6c8bz7_sm0000gn/T/ipykernel_26709/612045185.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/7v/5sz92nkn6s13fvy6c8bz7_sm0000gn/T/ipykernel_26709/612045185.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

       Time           X            Y       dirX        dirY      angles  \
3     0.048 -279.999056  3550.083823        NaN         NaN         NaN   
5     0.288 -280.004222  3549.625202  -0.005166   -0.458621         NaN   
7     0.528 -279.965389  3553.072657   0.038833    3.447455  179.999999   
9     0.768 -280.064521  3560.216660  -0.099133    7.144003    1.440369   
10    0.928 -280.314612  3565.704150  -0.250091    5.487490    1.814430   
..      ...         ...          ...        ...         ...         ...   
711  93.690 -424.000000   407.000000  15.000000  281.000000    0.038465   
713  93.930 -416.000000   536.000000   8.000000  129.000000    0.493089   
715  94.090 -412.000000   628.000000   4.000000   92.000000    1.059129   
717  94.330 -402.000000   815.000000  10.000000  187.000000    0.571477   
719  94.610 -389.000000  1081.000000  13.000000  266.000000    0.263086   

      BaseAngle  
3           NaN  
5      0.000000  
7    179.999999  
9    178.559631  
10   176.

,Speed,RPM,Throttle,Brake,nGear,X,Y,Z,Time,LapNumber,Source,dx,dy,dirX,dirY,BaseAngle,DeltaBaseAngle,angles
5,281,11150,100,False,7,-376.0,1357.0,-158.0,0.085,19.0,car,7.332498,148.790698,7.332498,148.790698,177.824076,0.067717,0.067717
7,283,11223,100,False,7,-366.0,1559.0,-159.0,0.365,19.0,car,10.000000,202.000000,10.000000,202.000000,177.811254,-0.012822,0.012822
9,284,11275,100,False,7,-357.0,1748.0,-158.0,0.605,19.0,car,9.000000,189.000000,9.000000,189.000000,177.919054,0.107800,0.107800
11,287,11303,100,False,7,-345.0,2041.0,-159.0,0.925,19.0,car,12.000000,293.000000,12.000000,293.000000,178.300091,0.381037,0.381037
14,288,11413,100,False,7,-335.0,2316.0,-159.0,1.325,19.0,car,10.000000,275.000000,10.000000,275.000000,178.562800,0.262709,0.262709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,270,10790,100,False,7,-430.0,283.0,-159.0,93.925,19.0,car,16.000000,301.000000,16.000000,301.000000,177.602606,-0.086448,0.086448
723,271,10806,100,False,7,-426.0,368.0,-159.0,94.085,19.0,car,4.000000,85.000000,4.000000,85.000000,177.951081,0.348475,0.348475
725,274,10913,100,False,7,-414.0,596.0,-158.0,94.405,19.0,car,12.000000,228.000000,12.000000,228.000000,177.632578,-0.318503,0.318503
727,276,11002,100,False,7,-403.0,814.0,-159.0,94.685,19.0,car,11.000000,218.000000,11.000000,218.000000,177.756743,0.124166,0.124166


In [110]:
speed_plot = px.scatter(speed_data_graph, x="Time", y="DeltaBaseAngle", color="Speed")

speed_plot.update_layout(
    plot_bgcolor="#2D2D2D",
    paper_bgcolor="#2D2D2D",
)

speed_plot

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Time=%{x}<br>DeltaBaseAngle=%{y}<br>Speed=%{marker.color}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': {'bdata': ('GQEbARwBHwEgASABIgEiASQBJAElAS' ... 'DyAPgA+QACAQYBCAEOAQ8BEgEUARcB'),
                                   'dtype': 'i2'},
                         'coloraxis': 'coloraxis',
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': ('w/UoXI/CtT9cj8L1KFzXP1yPwvUoXO' ... 'geheuZV0CkcD0K16tXQD0K16NwxVdA'),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': ('AFChKeZVsT8AwKeXTEKKvwA4t0LImL' ... 'JNTlti1L8AICsJV8m/PwA4qTak4rC/'),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Speed'}},
                             'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'paper_bgcolor': '#2D2D2D',
               'plot_bgcolor': '#2D2D2D',
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Time'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'DeltaBaseAngle'}}}
})